
# Flowsheet PFR Simulation and Optimization of Ethylene Glycol Production


## Learning outcomes


- Call and implement the IDAES PFR unit model
- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a flowsheet using Arcs
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Following the previous example implementing a CSTR unit model, we can alter the flowsheet to use a plug flow reactor (PFR). As before, this example is adapted from Fogler, H.S., Elements of Chemical Reaction Engineering 5th ed., 2016, Prentice Hall,  p. 157-160 with the following chemical reaction, property packages and flowsheet. Unlike a CSTR which assumes well-mixed liquid behavior, the concentration profiles will vary spacially in one dimension. In actuality, following start-up flow reactor exhibit dynamic behavior as they approach a steady-state equilibrium; we will assume our system has already achieved steady-state behavior. The state variables chosen for the property package are **molar flows of each component by phase in each stream, temperature of each stream and pressure of each stream**. The components considered are: **ethylene oxide, water, sulfuric acid and ethylene glycol** and the process occurs in liquid phase only. Therefore, every stream has 4 flow variables, 1 temperature and 1 pressure variable.

Chemical reaction:

**C<sub>2</sub>H<sub>4</sub>O</sub> + H<sub>2</sub>O</sub> + H<sub>2</sub>S</sub>O<sub>4</sub> → C<sub>2</sub>H<sub>6</sub>O<sub>2</sub> + H<sub>2</sub>S</sub>O<sub>4</sub>**

Property Packages:

- egprod_ideal.py
- egprod_reaction.py

Flowsheet:

![](egprod_flowsheet.png)


## Importing required Pyomo and IDAES components


To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- PFR

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom, tools for model expressions and calling variable values, and built-in functions to define property packages, add unit containers to objects and define our initialization scheme. Since PFR units do not exhibit well-mixed behavior, we need to import a DerivativeVar for our performance equation:


In [1]:
from pyomo.environ import (Constraint,
                           ConstraintList,
                           exp,
                           log,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           TransformationFactory,
                           value,
                           units as pyunits)
from pyomo.dae import DerivativeVar, ContinuousSet, Integral
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
from idaes.generic_models.properties.core.generic.generic_reaction import (
        GenericReactionParameterBlock)
from idaes.generic_models.unit_models import (Mixer,
                                              Heater,
                                              PFR)

from idaes.core.util.constants import Constants
from idaes.core.util import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

import idaes.logger as idaeslog

## Importing required thermo and reaction package

The final set of imports are to import the thermo and reaction package. We have created a custom thermo package that support ideal vapor and liquid behavior for this system, and in this case we will restrict it to ideal liquid behavior only.

The reaction package here assumes Arrhenius kinetic behavior for the PFR, for which $k_0$ and $E_a$ are known *a priori* (if unknown, they may be obtained using one of the parameter estimation tools within IDAES).

$ r = -kVC_{EO} $, $ k = k_0 e^{(-E_a/RT)}$, with the variables as follows:

$r$ - reaction rate extent in moles of ethylene oxide consumed per second; note that the traditional reaction rate would be given by $rate = r/V$ in moles per $m^3$ per second  
$k$ - reaction rate constant per second  
$V$ - volume of PFR in $m^3$, note that this is *liquid volume* and not the *total volume* of the reactor itself  
$C_{EO}$ - bulk concentration of ethylene oxide in moles per $m^3$ (the limiting reagent, since we assume excess catalyst and water)  
$k_0$ - pre-exponential Arrhenius factor per second  
$E_a$ - reaction activation energy in kJ per mole of ethylene oxide consumed  
$R$ - gas constant in J/mol-K  
$T$ - reactor temperature in K

These calculations are contained within the property, reaction and unit model packages, and do not need to be entered into the flowsheet. More information on property estimation may be found below:

ParamEst parameter estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/workflow/data_rec_parmest.html?highlight=paramest  
HELMET thermodynamic estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/modeling_extensions/surrogate/helmet/index.html  
RIPE reaction estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/modeling_extensions/surrogate/ripe/index.html  

Let us import the following modules from the same directory as this Jupyter notebook:
- egprod_ideal as thermo_props
- egprod_reaction as reaction_props

In [2]:
import egprod_ideal as thermo_props
import egprod_reaction as reaction_props

## Constructing the Flowsheet

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block. 

In [3]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. We will use the Generic Property and Generic Reaction Frameworks; more information may be found on these methods at https://idaes-pse.readthedocs.io/en/1.8.0/user_guide/components/property_package/index.html.

In [4]:
m.fs.thermo_params = GenericParameterBlock(default=thermo_props.config_dict)
m.fs.reaction_params = GenericReactionParameterBlock(default={"property_package": m.fs.thermo_params,
                                                              **reaction_props.config_dict})

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding a Mixer (assigned a name M101), a Heater (assigned a name H101) and a CSTR (assigned a name R101). Note that all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/model_libraries/core_lib/unit_models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the two inlets.

For the PFR, note that the last three arguments relate to the spacial discretization. They are written explicitly here for demonstration, and are taken as default values (20 elements using backwards finite difference) if not provided. The length domain set is assumed to be (0, 1) if not specified otherwise.

In [5]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["reagent_feed", "catalyst_feed"]})
m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": False})

In [6]:
m.fs.R101 = PFR(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False,
                     "transformation_method": "dae.finite_difference",
                     "transformation_scheme": "BACKWARD",
                     "finite_elements": 20,
                     "length_domain_set": [0, 1]})

## Connecting Unit Models using Arcs

We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101), and the outlet of the heater(H101) to the inlet of the reactor(R101).

In [7]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [8]:
TransformationFactory("network.expand_arcs").apply_to(m)

## Adding expressions to compute operating costs

In this section, we will add a few Expressions that allows us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

For this flowsheet, we are interested in computing ethylene glycol production in millions of pounds per year, as well as the total costs due to cooling and heating utilities:

Let us first add an Expression to convert the product flow from mol/s to MM lb/year of ethylene glycol. We see that our molecular weight exists in the thermo property package, so we may use that value for our calculations.

In [9]:
m.fs.eg_prod = Expression(expr=pyunits.convert(m.fs.R101.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]
                                               *m.fs.thermo_params.ethylene_glycol.mw, # MW defined in properties as kg/mol
                                               to_units=pyunits.Mlb/pyunits.yr)) # converting kg/s to MM lb/year

Now, let us add expressions to compute the reactor cooling cost (\\$/s) assuming a cost of 0.212E-4 \\$/kW, and the heating utility cost (\\$/s) assuming 2.2E-4 \\$/kW. Note that the heat duty is in units of watt (J/s). The total operating cost will be the sum of the two, expressed in \\$/year assuming 8000 operating hours per year (~10\% downtime, which is fairly common for small scale chemical plants):

In [10]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-sum(m.fs.R101.heat_duty[0, x]
                                                     for x in m.fs.R101.control_volume.length_domain)))  # the reaction is exothermic, so R101 duty is negative
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0])  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.operating_cost = Expression(expr=(3600 * 8000 *(m.fs.heating_cost + m.fs.cooling_cost)))

## Fixing feed conditions

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. We expect each stream to have 6 degrees of freedom, the mixer to have 0 (after both streams are accounted for), the heater to have 1 (just the duty, since the inlet is also the outlet of M101), and the reactor to have 2 unit specifications and 1 specification for each finite element. Therefore, we have 35 degrees of freedom to specify: temperature, pressure and flow of all four components on both streams; outlet heater temperature; a property of each finite element, reactor volume and reactor length.

In [11]:
print(degrees_of_freedom(m))

35


In [12]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 35

We will now be fixing the feed stream to the conditions shown in the flowsheet above. As mentioned in other tutorials, the IDAES framework expects a time index value for every referenced internal stream or unit variable, even in steady-state systems with a single time point $ t = 0 $. The non-present components in each stream are assigned a very small non-zero value to help with convergence and initializing. Based on stoichiometric ratios for the reaction, 80% conversion and 200 MM lb/year (46.4 mol/s) of ethylene glycol, we will initialize our simulation with the following calculated values:

In [13]:
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(58.0*pyunits.mol/pyunits.s)
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(39.6*pyunits.mol/pyunits.s)  # calculated from 16.1 mol EO / cudm in stream
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(1e-5*pyunits.mol/pyunits.s)
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-5*pyunits.mol/pyunits.s)
m.fs.M101.reagent_feed.temperature.fix(298.15*pyunits.K)
m.fs.M101.reagent_feed.pressure.fix(1e5*pyunits.Pa)

m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(1e-5*pyunits.mol/pyunits.s)
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(200*pyunits.mol/pyunits.s)
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(0.334*pyunits.mol/pyunits.s)  # calculated from 0.9 wt% SA in stream
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-5*pyunits.mol/pyunits.s)
m.fs.M101.catalyst_feed.temperature.fix(298.15*pyunits.K)
m.fs.M101.catalyst_feed.pressure.fix(1e5*pyunits.Pa)

## Fixing unit model specifications

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us fix the outlet temperature of H101 to 328.15 K. 

In [14]:
m.fs.H101.outlet.temperature.fix(328.15*pyunits.K)

We'll add constraints defining the reactor volume and conversion in relation to the stream properties. Particularly, we want to use our CSTR performance relation:  

$\frac{dX} {dV} = \frac{k} {v_0} (1-X)$, where the PFR reaction volume $V$ will be specified, the inlet volumetric flow $v_0$ is determined by stream properties, $k$ is calculated by the reaction package, and $X$ will be calculated. Reactor volume is commonly selected as a specification in simulation problems, and choosing conversion is often to perform reactor design. For simplicity, here we will index by the length domain and extract out the constant area variable: $\frac{dX} {dx} = A \frac{k} {v_0} (1-X)$

For the PFR, we have to define the conversion in terms of ethylene oxide as well as the PFR reaction volume. For simplicity, we'll define conversion as *total conversion* at each spacial point, e.g. $X(x=0.5)=0.5$ means 50% of the inlet reactant as been converted by the time the flow reaches halway through the reactor. Note that the PFR reaction volume variable (m.fs.R101.volume) and length domain set (m.fs.R101.control_volume.length_domain) do not need to be defined here since they are internally defined by the PFR model. Reactor length will be calculated from the specified conversion, and the cross-sectional area will be decided as the solver varies the volume.

We do, however, need to specify each finite element via a first-order reaction performance equation. This can be done succintly by adding a rate_constant(m, x) method to which we pass the model and each spacial point. Note that our performance equation is a differential equation, and we must index our differentiable variable (conversion) by a continuous set (our length domain). The heat duty is not fixed, since the heat of reaction depends on the reactor conversion, and temperature effects are captured as well by our rate constant method. We'll estimate 80% conversion for our initial flowsheet:

In [15]:
m.fs.R101.conversion = Var(m.fs.R101.control_volume.length_domain, bounds=(0,1),
                           initialize=0.80, units=pyunits.dimensionless)  # fraction

def conv_rule(R101, x):  # rule for calculating cumulative conversion at each finite element
    return (R101.conversion[x] * R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"] ==
            (R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"] -
            R101.control_volume.properties[0, x].flow_mol_phase_comp["Liq", "ethylene_oxide"]))
m.fs.R101.conv_constraint = Constraint(m.fs.R101.control_volume.length_domain, rule=conv_rule)


def rate_constant(m, x):  # rule for calculating reaction rate constant at every finite element
    return (m.fs.reaction_params.reaction_R1.arrhenius_const * exp(-m.fs.reaction_params.reaction_R1.energy_activation /
           (Constants.gas_constant*m.fs.R101.control_volume.properties[0, x].temperature)))  # r = k[EO], k = [/s]

m.fs.R101.dconv_dlength = DerivativeVar(m.fs.R101.conversion, wrt=m.fs.R101.control_volume.length_domain)

def perf_eqn_rule(m, x):  # rule for PFR performance equation at each finite element
    return (m.fs.R101.dconv_dlength[x] == m.fs.R101.control_volume.area * (rate_constant(m, x) /
                                                                           (2*3.62e-3)) * (1 - m.fs.R101.conversion[x]))
m.perf_eqn_constraints = Constraint(m.fs.R101.control_volume.length_domain,rule=perf_eqn_rule)

m.fs.R101.control_volume.length.fix(1*pyunits.m)
m.fs.R101.volume.fix(5.538*pyunits.m**3)

In [16]:
print(degrees_of_freedom(m))

20


In [17]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 0

AssertionError: 

Finally, we need to initialize the each unit operation in sequence to solve the flowsheet. In best practice, unit operations are initialized or solved, and outlet properties are propagated to connected inlet streams via arc definitions as follows:

In [ ]:
# Initialize and solve each unit operation
m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.R101.initialize(outlvl=idaeslog.DEBUG)

# set solver
solver = get_solver()

In [ ]:
# Solve the model
results = solver.solve(m, tee=True)

In [ ]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

## Analyze the results of the square problem


What is the total operating cost? 

In [ ]:
print('operating cost = $', value(m.fs.operating_cost), ' per year')

In [ ]:
import pytest
assert value(m.fs.operating_cost)/1E6 == pytest.approx(3.451440, abs=1e-3)

For this operating cost, what conversion did we achieve of ethylene oxide to ethylene glycol? 

In [ ]:
m.fs.R101.report()

print()
print('Conversion achieved = ', value(m.fs.R101.conversion)*100, '%')
print()
print('Assuming a 20% design factor for reactor volume, total CSTR volume required = ', value(1.2*m.fs.R101.volume),
      'm^3 = ', value(pyunits.convert(1.2*m.fs.R101.volume, to_units=pyunits.gal)), ' gal')

In [ ]:
assert value(m.fs.R101.conversion) == pytest.approx(0.79845, abs=1e-3)
assert value(m.fs.R101.volume) == pytest.approx(5.5380, abs=1e-3)
assert value(m.fs.R101.heat_duty[0,0])/1E6 == pytest.approx(-5.6457, abs=1e-3)
assert value(m.fs.R101.outlet.temperature[0])/1E2 == pytest.approx(3.2846, abs=1e-3)

## Optimizing Ethylene Glycol Production

Now that the flowsheet has been squared and solved, we can run a small optimization problem to minimize our production costs. Suppose we require at least 200 million pounds/year of ethylene glycol produced and at least 90% conversion of ethylene oxide, allowing for variable reactor volume (considering operating/non-capital costs only) and reactor temperature (heater outlet).

Let us declare our objective function for this problem. 

In [ ]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

Now, we need to add the design constraints and unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now, as well as set bounds for the design variables:

In [ ]:
m.fs.eg_prod_con = Constraint(expr=m.fs.eg_prod >= 200*pyunits.Mlb/pyunits.yr)  # MM lb/year
m.fs.conversion_con = Constraint(expr=m.fs.R101.conversion >= 0.90*pyunits.dimensionless)  # fraction

m.fs.R101.volume.unfix()
m.fs.R101.volume.setlb(0*pyunits.m**3)
m.fs.R101.volume.setub(pyunits.convert(5000*pyunits.gal, to_units=pyunits.m**3))

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.outlet.temperature[0].setlb(328.15*pyunits.K)
m.fs.H101.outlet.temperature[0].setub(470.45*pyunits.K)  # highest component boiling point (ethylene glycol)

In [ ]:
assert degrees_of_freedom(m) == 2


We have now defined the optimization problem and we are now ready to solve this problem. 




In [ ]:
results = solver.solve(m, tee=True)

In [ ]:
# Check for solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

In [ ]:
print('operating cost = $', value(m.fs.operating_cost), 'per year')

print()
print('Heater results')

m.fs.H101.report()

print()
print('CSTR reactor results')

m.fs.R101.report()

In [ ]:
assert value(m.fs.operating_cost)/1E6 == pytest.approx(3.889837, abs=1e-3)
assert value(m.fs.R101.volume) == pytest.approx(12.581, abs=1e-3)

Display optimal values for the decision variables and design variables:

In [ ]:
print('Optimal Values')
print()

print('H101 outlet temperature = ', value(m.fs.H101.outlet.temperature[0]), 'K')

print()
print('Assuming a 20% design factor for reactor volume, total CSTR volume required = ', value(1.2*m.fs.R101.volume[0]),
      'm^3 = ', value(pyunits.convert(1.2*m.fs.R101.volume, to_units=pyunits.gal)), ' gal')

print()
print('Ethylene glycol produced = ', value(m.fs.eg_prod), 'MM lb/year')

print()
print('Conversion achieved = ', value(m.fs.R101.conversion)*100, ' %')

In [ ]:
assert value(m.fs.H101.outlet.temperature[0])/100 == pytest.approx(3.2815, abs=1e-3)
assert value(m.fs.R101.volume) == pytest.approx(12.581, abs=1e-3)
assert value(m.fs.eg_prod) == pytest.approx(225.415, abs=1e-3)
assert value(m.fs.R101.conversion)*100 == pytest.approx(90.0, abs=1e-3)